## LSTM 셀상태(Cell State) 및 게이트(Gate) 메커니즘 이해하기

In [1]:
# sequence data 정의
import torch
import torch.nn as nn

sequence_data = torch.tensor([
    [25.0, 70.0, 10.0, 0.0],  # 시점 t-5
    [22.0, 80.0, 5.0, 0.0],   # 시점 t-4
    [15.0, 80.0, 20.0, 5.0],  # 시점 t-3
    [10.0, 90.0, 5.0, 20.0],  # 시점 t-2
    [5.0, 85.0, 30.0, 10.0],  # 시점 t-1
    [18.0, 75.0, 10.0, 0.0]])

print(sequence_data[0])
print(sequence_data[0].shape)
print(sequence_data.shape)

tensor([25., 70., 10.,  0.])
torch.Size([4])
torch.Size([6, 4])


LSTMcell 주요 변수 설정 및 초기화

In [2]:
# 첫번째 데이터 포인트
input_data = sequence_data[0].unsqueeze(0) # .unsqueeze(0) 0번 차원에 새로운 차원 추가
input_size = input_data.shape[1]

# 초기 은닉 상태와 셀 상태 설정
hidden_size = 10

h_0 = torch.zeros(1, hidden_size)
c_0 = torch.zeros(1, hidden_size)

In [4]:
print('입력 데이터 형태:', input_data.shape)
print('입력 데이터 (특성) 크기:', input_size)
print('은닉 상태 형태:', h_0.shape)
print('셀 상태 형태:', c_0.shape)

입력 데이터 형태: torch.Size([1, 4])
입력 데이터 (특성) 크기: 4
은닉 상태 형태: torch.Size([1, 10])
셀 상태 형태: torch.Size([1, 10])


망각 게이트(forget gate)

In [5]:
Wx_f = nn.Parameter(torch.randn(input_size, hidden_size))
Wh_f = nn.Parameter(torch.randn(hidden_size, hidden_size))
b_f = nn.Parameter(torch.randn(hidden_size))

# 망각게이트 계산
f_t = torch.sigmoid(torch.matmul(input_data, Wx_f) + torch.matmul(h_0, Wh_f) + b_f)

In [6]:
print('망각 게이트의 입력 가중치 행렬(Wx_f) 형태:', Wx_f.shape)
print('망각 게이트의 은닉상태 가중치 (Wh_f) 형태:', Wh_f.shape)
print('망각 게이트의 편향 벡터 (b_f) 형태:', b_f.shape)
print("f_t (망각 게이트 형태):", f_t.shape)

망각 게이트의 입력 가중치 행렬(Wx_f) 형태: torch.Size([4, 10])
망각 게이트의 은닉상태 가중치 (Wh_f) 형태: torch.Size([10, 10])
망각 게이트의 편향 벡터 (b_f) 형태: torch.Size([10])
f_t (망각 게이트 형태): torch.Size([1, 10])


입력게이트(input gate)

In [7]:
Wx_i = nn.Parameter(torch.randn(input_size, hidden_size))
Wh_i = nn.Parameter(torch.randn(hidden_size, hidden_size))
b_i = nn.Parameter(torch.randn(hidden_size))

# 입력게이트 계산
i_t = torch.sigmoid(torch.matmul(input_data, Wx_i) + torch.matmul(h_0, Wh_i) + b_i )

In [8]:
display('입력 게이트의 입력 가중치 행렬(Wx_i) 형태:', Wx_i.shape)
display('입력 게이트의 은닉상태 가중치 행렬(Wh_i) 형태:', Wh_i.shape)
display('입력 게이트의 편향 벡터 (b_i) 형태:', b_i.shape)
display("i_t (입력 게이트 형태):", i_t.shape)

'입력 게이트의 입력 가중치 행렬(Wx_i) 형태:'

torch.Size([4, 10])

'입력 게이트의 은닉상태 가중치 행렬(Wh_i) 형태:'

torch.Size([10, 10])

'입력 게이트의 편향 벡터 (b_i) 형태:'

torch.Size([10])

'i_t (입력 게이트 형태):'

torch.Size([1, 10])

후보 셀 상태(Candidate cell state)

In [9]:
# 후보 셀상태 혹은 셀게이트 계산
Wx_C = nn.Parameter(torch.randn(input_size, hidden_size))
Wh_C = nn.Parameter(torch.randn(hidden_size, hidden_size))
b_C = nn.Parameter(torch.randn(hidden_size))

# 후보 셀 상태 계산
C_hat_t = torch.tanh(torch.matmul(input_data, Wx_C) + torch.matmul(h_0, Wh_C) + b_C)

In [10]:
display('후보 셀상태의 입력 가중치 행렬(Wx_C) 형태:', Wx_C.shape)
display('후보 셀상태의 은닉상태 가중치 행렬(Wh_C) 형태:', Wh_C.shape)
display('후보 셀상태의 편향 벡터 (b_C) 형태:', b_C.shape)
display("C_hat_t (후보 셀상태 형태):", C_hat_t.shape)

'후보 셀상태의 입력 가중치 행렬(Wx_C) 형태:'

torch.Size([4, 10])

'후보 셀상태의 은닉상태 가중치 행렬(Wh_C) 형태:'

torch.Size([10, 10])

'후보 셀상태의 편향 벡터 (b_C) 형태:'

torch.Size([10])

'C_hat_t (후보 셀상태 형태):'

torch.Size([1, 10])

최종 셀 상태(cell state)


In [11]:
# 셀 상태 계산
cx = f_t * c_0 + i_t * C_hat_t
print(f"Cell state : {cx}")
print(f"Cell state shape: {cx.shape}")

Cell state : tensor([[-1.0000e+00,  2.8103e-38,  2.8118e-35,  8.9860e-01,  3.1119e-18,
          1.0000e+00,  0.0000e+00, -1.0000e+00,  2.4096e-30,  9.9998e-01]],
       grad_fn=<AddBackward0>)
Cell state shape: torch.Size([1, 10])


출력 게이트(output gate) 및 최종 은닉상태(hidden state)

In [14]:
Wx_o = nn.Parameter(torch.randn(input_size, hidden_size ))
Wh_o = nn.Parameter(torch.randn(hidden_size, hidden_size))
b_o = nn.Parameter(torch.randn(hidden_size))

# 출력  게이트 계산
o_t = torch.sigmoid(torch.matmul(input_data, Wx_o ) + torch.matmul(h_0, Wh_o) + b_o)

# 은닉 상태 계산
hx = o_t * torch.tanh(cx)
print(f"Hidden state shape: {hx.shape}")
print(f"output: {o_t}")

Hidden state shape: torch.Size([1, 10])
output: tensor([[1.0000e+00, 5.7978e-07, 8.8222e-07, 1.0000e+00, 2.6497e-32, 9.3088e-16,
         8.8081e-01, 1.0000e+00, 1.0000e+00, 0.0000e+00]],
       grad_fn=<SigmoidBackward0>)


In [15]:
display('출력 게이트의 입력 가중치 행렬(Wx_o) 형태:', Wx_o.shape)
display('출력 게이트의 은닉상태 가중치 행렬(Wh_o) 형태:', Wh_o.shape)
display('출력 게이트의 편향 벡터 (b_o) 형태:', b_o.shape)
display("o_t (출력 게이트 형태):", o_t.shape)

'출력 게이트의 입력 가중치 행렬(Wx_o) 형태:'

torch.Size([4, 10])

'출력 게이트의 은닉상태 가중치 행렬(Wh_o) 형태:'

torch.Size([10, 10])

'출력 게이트의 편향 벡터 (b_o) 형태:'

torch.Size([10])

'o_t (출력 게이트 형태):'

torch.Size([1, 10])

종합 코드

In [16]:
import torch
import torch.nn as nn

sequence_data = torch.tensor([
    [25.0, 70.0, 10.0, 0.0],  # 시점 t-5
    [22.0, 80.0, 5.0, 0.0],    # 시점 t-4
    [15.0, 80.0, 20.0, 5.0], # 시점 t-3
    [10.0, 90.0, 5.0, 20.0], # 시점 t-2
    [5.0, 85.0, 30.0, 10.0],  # 시점 t-1
    [18.0, 75.0, 10.0, 0.0]   # 시점 t
])

# 첫 번째 데이터 포인트
input_data = sequence_data[0].unsqueeze(0)  # n_sequence = 1
input_size = input_data.shape[1]

# 초기 은닉 상태와 셀 상태 설정
hidden_size = 10  # 은닉 상태의 크기 설정

h_0 = torch.zeros(1, hidden_size)
c_0 = torch.zeros(1, hidden_size)

# 망각  게이트
Wx_f = nn.Parameter(torch.randn(input_size,hidden_size))
Wh_f = nn.Parameter(torch.randn(hidden_size, hidden_size))
b_f = nn.Parameter(torch.randn(hidden_size))

f_t = torch.sigmoid(torch.matmul(input_data, Wx_f) + torch.matmul(h_0, Wh_f ) + b_i)

# 입력  게이트
Wx_i = nn.Parameter(torch.randn(input_size, hidden_size ))
Wh_i = nn.Parameter(torch.randn(hidden_size, hidden_size))
b_i = nn.Parameter(torch.randn(hidden_size))

i_t = torch.sigmoid(torch.matmul(input_data, Wx_i) + torch.matmul( h_0, Wh_i) + b_i)

# 후보 셀상태
Wx_C = nn.Parameter(torch.randn(input_size, hidden_size))
Wh_C = nn.Parameter(torch.randn(hidden_size, hidden_size))
b_C = nn.Parameter(torch.randn(hidden_size))

C_hat_t = torch.tanh(torch.matmul(input_data, Wx_C) + torch.matmul(h_0, Wh_C) + b_C)

# 최종 셀 상태 계산
cx = torch.mul(f_t, c_0) + torch.mul(i_t, C_hat_t)

# 출력  게이트 계산
Wx_o = nn.Parameter(torch.randn(input_size, hidden_size ))
Wh_o = nn.Parameter(torch.randn(hidden_size, hidden_size))
b_o = nn.Parameter(torch.randn(hidden_size))
o_t = torch.sigmoid(torch.matmul(input_data,Wx_o ) + torch.matmul(h_0, Wh_o) + b_o)

# 은닉 상태 계산
hx = o_t * torch.tanh(cx)
print(f"cell state shape: {cx.shape}")
print(f"Hidden state shape: {hx.shape}")

cell state shape: torch.Size([1, 10])
Hidden state shape: torch.Size([1, 10])
